# Example 03: Exploring Maze
In this example, we will demonstrate the capabilities of the `libmata` library on solving simple mazes.

#### Setup
We will use `libmata` and `OnTheFlyAlphabet` as shared alphabet through the example, though our alphabet will have only `4` different symbols (directions of walk in the maze).

Further we will use `random` to generate random maze, and `operator` and `itertools` for certain code functions.

In [1]:
import libmata as mata
import random
import operator
import itertools
mata.store()['alphabet'] = mata.OnTheFlyAlphabet()

#### Helper functions
First we will define three helper functions for generating random maze and plotting the maze as ascii picture.

Note, you can adjust the `wall_ratio` to generate either less (increase the ration) or more (decrease the ratio) walls.

In [2]:
wall_ratio = 3
def random_room() -> tuple[bool, bool]:
    """Generate walls in the given room. 
    
    E.g. the pair (True, True), means that we can go from the room both right and down.
    
    :return: random pair of bools that represents whether we can go right or down in the given room
    """
    return random.randint(0, wall_ratio) > 0, random.randint(0, wall_ratio) > 0

def generate_maze(width: int, height: int) -> list[list[int]]:
    """Generates random maze represented as list of rooms. Each room is a pair of bools that define
    whether we can travel right or down in the maze.
    
    :param int width: width of the maze
    :param int height: height of the maze
    :return: maze represented as list of list of rooms
    """
    return [
        [random_room() for _ in range(0, width)]
        for _ in range(0, height)
    ]

def plot_maze(maze: list[list], path_to_goal: list = None):
    """Plots the maze. If path_to_goal is set as list of visited rooms, the path in the maze towards
    exit will be highlighted.
    
    :param list maze: maze represented as list of list of rooms
    :param list path_to_goal: optional path to the exist
    """
    width = len(maze[0])
    height = len(maze)
    cells = [[' ' for _ in range(0, len(maze[0]))] for _ in range(0, len(maze))]
    if path_to_goal:
        for start, end in zip(path_to_goal, path_to_goal[1:]):
            start_x, start_y = start // width, start % width
            end_x, end_y = end // width, end % width
            if start_x < end_x:
                cells[start_x][start_y] = "↓"
            elif start_x > end_x:
                cells[start_x][start_y] = "↑"
            elif start_y < end_y:
                cells[start_x][start_y] = "→"
            elif start_y > end_y:
                cells[start_x][start_y] = "←"
        cells[-1][-1] = "↓"
            
    result = "┏" + ("↓" if path_to_goal else " ") + "┳━"*(len(maze[0])-1) + "┓\n"
    for i, rooms in enumerate(maze):
        right_passages = ['┃' if not a else ' ' for a in map(operator.itemgetter(0), rooms)][:-1]
        row = [x for x in itertools.chain(*itertools.zip_longest(cells[i], right_passages)) if x is not None]
        result += '┃' + "".join(row) + "┃\n"
        if i < len(maze) - 1:
            bottom_passages = ['━' if not a else ' ' for a in (map(operator.itemgetter(1), rooms))]
            result += '┣' + "╋".join(bottom_passages) + "┫\n"
    result += "┗" + "━┻"*(len(maze[0])-1) + ("↓" if path_to_goal else " ") + "┛\n"
    print(result)

#### Generating maze
We will generate small maze, then transform it to automaton and try to find, if the language is not empty. If the language is not empty, it will give us a counterexample path to from initial state (start) to final state (goal).

Note that we start at top left corner, and want to get to bottom right corner.

In [3]:
width, height = 10, 3
small_maze = generate_maze(width, height)

In [4]:
plot_maze(small_maze)

┏ ┳━┳━┳━┳━┳━┳━┳━┳━┳━┓
┃     ┃         ┃ ┃ ┃
┣ ╋ ╋ ╋ ╋━╋ ╋━╋ ╋ ╋━┫
┃       ┃   ┃   ┃   ┃
┣ ╋ ╋━╋ ╋ ╋ ╋━╋ ╋ ╋ ┫
┃     ┃             ┃
┗━┻━┻━┻━┻━┻━┻━┻━┻━┻ ┛



#### Transforming maze to automaton
We will model each room in the maze as a state in the automaton. In particular, state `i` will correspond to the position `(i // width, i % width)` in the automaton.

For simplicity in each room we will only focus on two walls: bottom and right ones. Note, that this sufficies, since we start from the top, where we cannot escape to left or up, and hence, for each subsequent room, the top and left walls will be solved from previous rooms.

For each room, we construct transitions to the next room (next from bottom, or next from right) as follows:

  1. If we generated door to the right, we construct transition to next room at right (with symbol `🡆`) and we also generate transition back to the current room (with symbol `🡄`).
  2. If we generated door to the bottom, we construct transition to the next room at bottom (with symbol `🡇`) and we also generate transition back to the current room (with symbol `🡅`).
  3. If we are at the bottom or rightmost corner, we do not generate the appropriate transitions.
    
 We mark the starting room as initial state, and final room as final state.
  
  

In [5]:
def to_automaton(maze):
    """Transform maze to automaton.
    
    For each non-wall section, we generate transition between rooms (states). The rooms can be traversed
    back and forth.
    
    :param list maze: list of list of rooms
    :return: maze represented as automaton
    """
    width = len(maze[0])
    height = len(maze)
    automaton_maze = mata.Nfa(width * height)
    automaton_maze.make_initial_state(0)
    for i, rooms in enumerate(maze):
        for j, room in enumerate(rooms):
            current_room = i * width + j
            next_room = current_room + 1
            bottom_room = current_room + width
            if j != width -1 and room[0]:
                automaton_maze.add_transition(current_room, "🡆", next_room)
                automaton_maze.add_transition(next_room, "🡄", current_room)
            if i != height -1 and room[1]:
                automaton_maze.add_transition(current_room, "🡇", bottom_room)
                automaton_maze.add_transition(bottom_room, "🡅", current_room)
    automaton_maze.make_final_state(width*height-1)
    return automaton_maze
automaton_maze = to_automaton(small_maze)

In [6]:
mata.plot(automaton_maze)

#### Solving the maze
The maze is solvable, if there exist path from initial state (starting room) to final state (final room), i.e., if the language of the automaton is not empty.

We try to solve the room, and plot the maze again, now displaying the path to the finish.

In [7]:
solution = mata.Run()
is_empty = mata.Nfa.is_lang_empty(automaton_maze, solution)
is_empty

False

In [8]:
plot_maze(small_maze, path_to_goal=solution.path)

┏↓┳━┳━┳━┳━┳━┳━┳━┳━┳━┓
┃→ → ↓┃         ┃ ┃ ┃
┣ ╋ ╋ ╋ ╋━╋ ╋━╋ ╋ ╋━┫
┃    → ↓┃   ┃   ┃   ┃
┣ ╋ ╋━╋ ╋ ╋ ╋━╋ ╋ ╋ ┫
┃     ┃→ → → → → → ↓┃
┗━┻━┻━┻━┻━┻━┻━┻━┻━┻↓┛



#### Solving more complex mazes
Now, we will try some more bigger maze. Note that we will omit the display automaton, only the maze itself and the path leading to exit (if we generated solvable maze).

In [9]:
bwidth, bheight = 60, 10
big_maze = generate_maze(bwidth, bheight)
big_automaton_maze = to_automaton(big_maze)
is_empty = mata.Nfa.is_lang_empty(big_automaton_maze, solution)
if not is_empty:
    plot_maze(big_maze, path_to_goal=solution.path)
else:
    print("Maze is not solvable")

┏↓┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┓
┃→ → → → → ↓┃   ┃     ┃   ┃         ┃ ┃   ┃       ┃ ┃           ┃   ┃           ┃       ┃ ┃   ┃     ┃     ┃   ┃   ┃     ┃
┣━╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋━╋ ╋ ╋━╋ ╋━╋ ╋━╋ ╋ ╋ ╋━╋━╋━╋ ╋ ╋ ╋━╋ ╋━╋━╋ ╋ ╋ ╋ ╋ ╋━╋━╋ ╋ ╋ ╋━╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋━╋━╋ ╋ ╋ ╋ ┫
┃ ┃        → → ↓┃     ┃           ┃ ┃ ┃   ┃   ┃ ┃                             ┃   ┃         ┃ ┃                   ┃     ┃
┣ ╋━╋ ╋━╋ ╋ ╋━╋ ╋ ╋ ╋━╋━╋━╋ ╋ ╋ ╋ ╋ ╋ ╋━╋ ╋━╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋━╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋━╋ ╋ ╋ ╋ ╋━╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋━╋ ╋ ╋━╋ ┫
┃     ┃   ┃    → → → → → ↓┃         ┃ ┃ ┃ ┃       ┃ ┃   ┃       ┃       ┃ ┃ ┃                               ┃       ┃   ┃
┣━╋ ╋ ╋ ╋━╋ ╋━╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋━╋━╋ ╋━╋━╋ ╋ ╋ ╋ ╋ ╋━╋ ╋ ╋━╋ ╋━╋ ╋ ╋ ╋ ╋━╋ ╋ ╋ ╋ ╋━╋━╋ ╋ ╋ ╋━╋━╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ┫
┃         ┃         ┃   ┃→ ↓┃      → ↓┃   ┃     ┃ ┃                   ┃ ┃       ┃ ┃   ┃           ┃           ┃   ┃ ┃   ┃
┣ ╋ ╋ ╋ ╋ ╋ ╋ ╋ ╋━╋ ╋ ╋ 